## Description

##### Implementation of "Deep Residual Learning for Image Recognition" paper - https://ieeexplore.ieee.org/document/7780459

##### Implementation of residual block

### Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import typing

In [ ]:
from ipynb.fs.full.residual_learning_layers import *

### Input

##### Let's assume we have an input image of [batch_size, image_width, image_height, features]

In [ ]:
input_shape = (1, 500, 500, 3) # images of batch 100, size of 500x500 and rgb hene feaures of size 3
x = tf.random.normal(input_shape)
x = tf.constant(x, dtype=tf.float32)

In [ ]:
x.shape

### Residual Block

In [ ]:
class ResidualBlock(tf.Module):
    
    def __init__(self, 
                out_channels: int, # aka nr. filters
                kernel: int, # the h*w of filter, can be a tuple but for now let's set it to a sqaure
                stride_1: int, # the h_steps*w_steps of filter as it moves along input, set to a square for now
                stride_2: int,
                name = None):
        super(ResidualBlock, self).__init__(name)
        
        self.conv2d_layer_1 = Conv2D(out_channels=out_channels,
                                     kernel=kernel,
                                     stride=stride_1)
        self.conv2d_layer_2 = Conv2D(out_channels=out_channels,
                                     kernel=kernel,
                                     stride=stride_2)
        self.batch_normalization = tf.keras.layers.BatchNormalization()
        
    def __call__(self, x_in: tf.Tensor):
        
        x = self.conv2d_layer_1(x_in)
        x = self.batch_normalization(x)
        x = tf.nn.leaky_relu(x)
        x = self.conv2d_layer_2(x)
        x = self.batch_normalization(x)
        x = tf.nn.leaky_relu(x)
        
        if x.shape != x_in.shape: # input should be projected (using 1x1 conv2d layer with stride of 2) to match dimensions of F(x_in)
            conv2d_proj_layer = Conv2D(out_channels=x.shape[-1], kernel=1, stride=2)
            x_in = conv2d_proj_layer(x_in)
    
        x = tf.add(x, x_in) # for residual effect, add input to output for final output
        
        return x

### Test Residual Block

In [ ]:
# setup prior layers to first residual block
conv2d_layer = Conv2D(out_channels=64, kernel=7, stride=2)
x_1 = conv2d_layer(x)

max_pool = tf.keras.layers.MaxPool2D(pool_size=2)
x_2 = max_pool(x_1)

In [ ]:
# test first residual block
residual_block = ResidualBlock(out_channels=64, 
                               kernel=3, 
                               stride_1=1, 
                               stride_2=1)
residual_block_output = residual_block(x_2)
residual_block_output.shape

In [ ]:
# test propagation of first residual block to second (testing dimensionality matching)
residual_block = ResidualBlock(out_channels=128, 
                               kernel=3, 
                               stride_1=2, 
                               stride_2=1)
residual_block_output_2 = residual_block(residual_block_output)
residual_block_output_2.shape